## Usage
To add short-term memory (thread-level persistence) to an agent, you need to specify a checkpointer when creating an agent.

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    "gpt-5",
    [get_user_info],
    checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

数据库持久化参看 [langgraph.checkpoint.postgres](https://reference.langchain.com/python/langgraph/checkpoints/#langgraph.checkpoint.postgres) ，目前似乎只支持 Postgre 数据库

## Customizing agent memory
为本轮对话加入一些预设的信息。      
由于 memory 也被看作 Agent 状态的 一部分来管理，因此 使用 from langchain.agents import create_agent, AgentState 来管理：        
这个例子 表明了 用户 喜欢 暗色主题

In [ ]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver


class CustomAgentState(AgentState):  
    user_id: str
    preferences: dict

agent = create_agent(
    "gpt-5",
    [get_user_info],
    state_schema=CustomAgentState,  
    checkpointer=InMemorySaver(),
)

# Custom state can be passed in invoke
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Hello"}],
        "user_id": "user_123",  
        "preferences": {"theme": "dark"}  
    },
    {"configurable": {"thread_id": "1"}})

## Common patterns
当对话长度超过模型支持的上下文，我们一般采用以下几种方法（参看 包： from langchain.messages）：
- Trim messages：减掉最初的 N 条Messages，在 Call 模型之前执行
- Delete messages：从 the graph state 中删除 messages ，用来删除 制定的具体的 某些信息
- Summarize messages：前两条方法都可能遗漏信息 | 使用 中间件 SummarizationMiddleware 实现

## Access memory
LangChain 提供了一系列方式去 访问和修改 short-term memory (state)
- Tools
- Prompt
- Before model | After model | 中间件

### 访问
Access short term memory (state) in a tool using the ToolRuntime parameter. tool_runtime 这个参数 是 hidden 的，模型看不到，不必理会。
### 修改
通过运行时 拿到相关 的信息后，from langgraph.types import Command 写一个命令来修改。是 字典 的形式：

````py
return Command(update={  
        "user_name": name,
        # update the message history
        "messages": [
            ToolMessage(
                "Successfully looked up user information",
                tool_call_id=runtime.tool_call_id
            )
        ]
    })
````